<a href="https://colab.research.google.com/github/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/img/Banner_Bootcamp.png?raw=true">

#**Projeto Final de Conclusão de Curso**
###Bootcamp Data Science Aplicada 2 by [Alura](https://www.alura.com.br/) 
####Autor: Helton Cordeiro
e-mail: heltoncordeiro@gmail.com

Junho-Agosto/2021.



---

# **Objetivo desse notebook**

Após os dados disponibilizados pelo Hospital Sírio Libânes ter sido tratados em [notebook próprio](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb) e termos trabalhando com alguns modelos de Machine Learning com seus valores default, vamos explorar os hiperparametros nesse notebook. 

Será utilizando um dicionário com o range de valores e parâmetros  que serão passados para o método **RandomizedSearchCV** utilizando a função *executa_modelos_RandomizedSearchCV* que está no arquivo funcoes.py.

A proposta da separação do [notebook de análise](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/Bootcamp_DataScience_projeto_final.ipynb) se deve ao tempo de processamento necessário para testar todos os modelos, além de facilitar futuros ajustes.

---


##O que é um hiperparâmetro?

Os hiperparâmetros contêm os dados que controlam o próprio processo de treinamento.

Seu aplicativo de treinamento lida com três categorias de dados durante o treinamento do modelo:

* Os *dados de entrada*, também chamados de dados de treinamento, formam uma coleção de registros individuais (instâncias) com as características que são importantes para o problema de machine learning. Esses dados configuram o modelo durante o treinamento para fazer predições precisas sobre novas instâncias de dados semelhantes. No entanto, os valores nos dados de entrada nunca se tornam diretamente parte do modelo.

* Os *parâmetros* do modelo são as variáveis que a técnica de machine learning escolhida usa para ajustar os dados. Por exemplo, uma rede neural profunda (DNN, na sigla em inglês) é composta por nós de processamento (neurônios), cada um com uma operação realizada nos dados enquanto eles trafegam pela rede. Quando a DNN é treinada, cada node tem um valor de peso que informa ao modelo o impacto que ele tem na predição final. Esses pesos são um exemplo dos parâmetros do modelo. De muitas formas, esses parâmetros são o modelo. Ou seja, são eles que diferenciam seu modelo específico de outros modelos do mesmo tipo que trabalham com dados semelhantes.

* Os *hiperparâmetros* são variáveis que controlam o próprio processo de treinamento. Por exemplo, faz parte da configuração de uma rede neural profunda decidir quantas camadas ocultas de nós precisam ser usadas entre a camada de entrada e a camada de saída, bem como quantos nós cada camada precisa usar. Essas variáveis não estão diretamente relacionadas aos dados de treinamento. Elas são variáveis de configuração. Os parâmetros mudam durante um job de treinamento, enquanto os hiperparâmetros geralmente permanecem constantes durante um job.


Os parâmetros do modelo são otimizados (ou seja, "ajustados") pelo processo de treinamento. Você executa os dados por meio das operações do modelo, compara a predição resultante com o valor real de cada instância de dados, avalia a precisão e ajusta até encontrar os melhores valores. Os hiperparâmetros são ajustados por meio da execução de todo o job de treinamento, a observação da precisão agregada e o ajuste. Nos dois casos, você está modificando a composição do modelo tentando encontrar a melhor combinação para lidar com o problema.

Fonte: [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)


##Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import random
from scipy.stats import randint

import warnings
warnings.simplefilter(action='ignore')

## Importando as funções auxiliares

* Arquivo funcoes.py contém as funções python que serão utilizados pelo projeto.

In [2]:
#!python funcoes.py
from funcoes import executa_modelos_RandomizedSearchCV

## Carregando os dados
Vamos carregar o arquivo de dados tratados e sem as colunas com alta correção.

O processo de tratamento dos dados estão nesse notebook [projeto_final_tratamento_dados.ipynb](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb).


### Realizando a carga de dados

In [3]:
url='https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/dados/Kaggle_Sirio_Libanes_ICU_Prediction-tratado-sem-corr.xls?raw=true'
dados_raw = pd.read_excel(url, index_col=0)
dados = dados_raw.copy()



---


#Modelos de Machine Learning
Relação dos Modelos que serão testados nesse projeto, foram selecionados por resolverem problemas de classificação binária e aprendizado supervisionado.


1. [KNeighborsClassifier](
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

2. [SVC - Linear](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

3. [SVC - RBF](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

4. [GaussianProcessClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html)

5. [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

6. [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

7. [Neural Net (MLP)](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

8. [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

9. [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

10. [ExtraTreesClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)



---



##Modelos

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

names = ["KNeighbors", "LinearSVC", "RBFSVC", "Gaussian",
         "DecisionTree", "RandomForest", "NeuralMLP", "AdaBoost",
         "Gradient", "ExtraTrees"]

classes = [
    KNeighborsClassifier(),
    SVC(kernel="linear", probability=True),
    SVC(kernel="rbf", probability=True),
    GaussianProcessClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    ExtraTreesClassifier() ]

##Testando Hiperparâmetros

Definindo o dicionário de parâmetros a serem testados.

In [5]:
hiperparams = { 
    "KNeighbors"  : { 
        "n_neighbors" : randint(2, 20),
        "weights" : ["uniform", "distance"],
        "leaf_size" : randint(25, 100),
        "metric" : ["minkowski","wminkowski", "euclidean"] 
    },
    "LinearSVC"   : { 
        "gamma" : ["scale","auto"],
        "C": randint(0,2)
    },
    "RBFSVC"      : { 
        "gamma" : ["scale","auto"],
        "C": randint(0,2)
    },
    "Gaussian"    : {
        "n_restarts_optimizer" : randint(0, 5),
        "max_iter_predict" : randint(50,500),
        "warm_start" : [False, True] 
    },
    "DecisionTree": {
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(2, 10),
        "min_samples_split" : randint(2, 129),
        "min_samples_leaf" : randint(1, 129)
    },
    "RandomForest": {
        "n_estimators" :randint(50, 500),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(3, 200),
        "min_samples_split" : randint(2, 300),
        "min_samples_leaf" : randint(1, 300),
        "bootstrap" : [True, False],
        "warm_start" : [False, True]
    },
    "NeuralMLP"   : {
        "activation" : ["identity", "logistic", "tanh", "relu"],
        "solver" : ["lbfgs", "sgd", "adam"],
        "shuffle" : [True, False],
        "warm_start" : [False, True]
    },
    "AdaBoost"    : {
        "n_estimators" : randint(25, 200),
        "algorithm" : ["SAMME","SAMME.R"]
    },
    "Gradient"    : {
        "learning_rate" : randint(0.1, 2),
        "n_estimators" : randint(50,500), 
        "min_samples_split" : randint(2, 25),
        "min_samples_leaf" : randint(1, 10), 
        "max_depth" : randint(1,15), 
        "warm_start" : [False, True],
        "subsample" : randint(0.5, 5)
    },
    "ExtraTrees"  : {
        "n_estimators" :randint(50, 500),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(0, 10),
        "min_samples_split" : randint(2, 129),
        "min_samples_leaf" : randint(1, 129),
        "bootstrap" : [True, False],
        "warm_start" : [False, True] 
    }
}

##Vamos processar as modelos utilizando 5 splits de dados, com 10 repetições e 15 iterações.

In [6]:
dfmodelosRand = executa_modelos_RandomizedSearchCV(names, classes, dados, 5, 10, hiperparams, 15)

Modelo: KNeighbors 	 tempo: 23 segundos
Modelo: LinearSVC 	 tempo: 27 segundos
Modelo: RBFSVC 	 tempo: 32 segundos
Modelo: Gaussian 	 tempo: 83 segundos
Modelo: DecisionTree 	 tempo: 12 segundos
Modelo: RandomForest 	 tempo: 238 segundos
Modelo: NeuralMLP 	 tempo: 420 segundos
Modelo: AdaBoost 	 tempo: 197 segundos
Modelo: Gradient 	 tempo: 12 segundos
Modelo: ExtraTrees 	 tempo: 334 segundos


In [7]:
dfmodelosRand

,Modelo,Accuracy,ROC AUC,PR AUC,Train ROC AUC,Best Params,Tempo
Nome,,,,,,,
RandomForest,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.665985,0.778032,0.671619,0.869315,"{'bootstrap': True, 'criterion': 'entropy', 'm...",238
ExtraTrees,"(ExtraTreeClassifier(ccp_alpha=0.0, class_weig...",0.643857,0.753486,0.636728,0.817983,"{'bootstrap': True, 'criterion': 'entropy', 'm...",334
NeuralMLP,"MLPClassifier(activation='logistic', alpha=0.0...",0.685663,0.749587,0.611202,0.812089,"{'warm_start': True, 'solver': 'adam', 'shuffl...",420
LinearSVC,"SVC(C=1, break_ties=False, cache_size=200, cla...",0.711905,0.741760,0.637487,0.849426,"{'C': 1, 'gamma': 'auto'}",27
RBFSVC,"SVC(C=1, break_ties=False, cache_size=200, cla...",0.650351,0.723472,0.570662,0.777884,"{'C': 1, 'gamma': 'scale'}",32
AdaBoost,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.695582,0.722144,0.617496,0.968695,"{'algorithm': 'SAMME', 'n_estimators': 57}",197
DecisionTree,"DecisionTreeClassifier(ccp_alpha=0.0, class_we...",0.703811,0.718168,0.551857,0.809147,"{'criterion': 'gini', 'max_depth': 5, 'min_sam...",12
Gaussian,"GaussianProcessClassifier(copy_X_train=True, k...",0.633326,0.712328,0.520752,0.950303,"{'max_iter_predict': 483, 'n_restarts_optimize...",83
KNeighbors,"KNeighborsClassifier(algorithm='auto', leaf_si...",0.645915,0.704664,0.528747,0.754837,"{'leaf_size': 29, 'metric': 'minkowski', 'n_ne...",23


# Lista dos melhores parâmetros

Após aproximadamente 23 minutos de processamentos temos a lista dos melhores modelos e parâmetros de acordo com as configurações apresentadas no dicionário ```hiperparams```.

In [8]:
for indice in names:
   display(indice, dfmodelosRand.at[indice,'Best Params'])

'KNeighbors'

{'leaf_size': 29,
 'metric': 'minkowski',
 'n_neighbors': 18,
 'weights': 'uniform'}

'LinearSVC'

{'C': 1, 'gamma': 'auto'}

'RBFSVC'

...

{'algorithm': 'SAMME', 'n_estimators': 57}

'Gradient'

{'learning_rate': 1,
 'max_depth': 10,
 'min_samples_leaf': 3,
 'min_samples_split': 19,
 'n_estimators': 215,
 'subsample': 1,
 'warm_start': True}

'ExtraTrees'

{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 9,
 'min_samples_leaf': 8,
 'min_samples_split': 84,
 'n_estimators': 248,
 'warm_start': False}

In [9]:
for indice in names:
   display(indice, dfmodelosRand.at[indice,'Modelo'])

'KNeighbors'

KNeighborsClassifier(algorithm='auto', leaf_size=29, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=18, p=2,
                     weights='uniform')

'LinearSVC'

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

'RBFSVC'

...

AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=1.0,
                   n_estimators=57, random_state=None)

'Gradient'

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=1, loss='deviance', max_depth=10,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=3, min_samples_split=19,
                           min_weight_fraction_leaf=0.0, n_estimators=215,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1, tol=0.0001,
                           validation_fraction=0.1, verbose=0, warm_start=True)

'ExtraTrees'

ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                     criterion='entropy', max_depth=9, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=8, min_samples_split=84,
                     min_weight_fraction_leaf=0.0, n_estimators=248,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)



---


## Referências

* [Kaggle - COVID-19 - Clinical Data to assess diagnosis - Sírio Libanês](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19)
* [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
* [SciKit Learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Configurar o treinamento do AutoML com Python](https://docs.microsoft.com/pt-br/azure/machine-learning/how-to-configure-auto-train)
* [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)
